In [1]:
%pip install -q trino

Note: you may need to restart the kernel to use updated packages.


In [1]:
CATALOG_URL = "http://lakekeeper:8181/catalog"
TRINO_URI = "https://trino-proxy:443"
KEYCLOAK_TOKEN_ENDPOINT = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"
WAREHOUSE = "bronze"

CLIENT_ID = "trino"
CLIENT_SECRET = "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ"

# Connect to Trino
In this example, Trino is configured to also use oauth. We are now logging into trino as a human user `hung`.
The following cell should print a link. Open this link in a new tab and login as:
* User: `hung`
* Password: `hung`

In [8]:
from trino.dbapi import connect
from trino.auth import OAuth2Authentication
from redirect_handler import REDIRECT_HANDLER
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

conn = connect(
    host=TRINO_URI,
    auth=OAuth2Authentication(REDIRECT_HANDLER),
    http_scheme="https",
    verify=False,
)

In [10]:
cur = conn.cursor()
print("\nResult of 'SELECT 1: ", cur.execute("Select 1").fetchall())
print("Result of 'SELECT current_user': ", cur.execute("SELECT current_user").fetchall())
# The current_user has now the unique ID from our IdP!

Open the following URL in browser for the external authentication:
http://localhost:38191/oauth2/token/initiate/d31bf8f58d78999999296488220ae7ba447750c3b3cd596da84391150f269c6c

Result of 'SELECT 1:  [[1]]
Result of 'SELECT current_user':  [['hung']]


# Creating the Trino Catalog
Make sure to follow the instructions in the `01-Bootstrap.ipynb` and `02-Create-Warehouse.ipynb` notebooks first!

In [ ]:
cur = conn.cursor()
cur.execute(
    f"""
    CREATE CATALOG lakekeeper USING iceberg
    WITH (
        "iceberg.catalog.type" = 'rest',
        "iceberg.rest-catalog.uri" = '{CATALOG_URL}',
        "iceberg.rest-catalog.warehouse" = '{WAREHOUSE}',
        "iceberg.rest-catalog.security" = 'OAUTH2',
        "iceberg.rest-catalog.oauth2.credential" = '{CLIENT_ID}:{CLIENT_SECRET}',
        "iceberg.rest-catalog.vended-credentials-enabled" = 'true',
        "iceberg.rest-catalog.oauth2.scope" = 'lakekeeper',
        "iceberg.rest-catalog.oauth2.server-uri" = '{KEYCLOAK_TOKEN_ENDPOINT}',
        "s3.region"= 'dummy',
        "s3.path-style-access" = 'true',
        "s3.endpoint" = 'http://minio:9000',
        "fs.native-s3.enabled" = 'true'
    )
    """
)